In [34]:
# Imports 
import unidecode
import pandas as pd
import numpy as np
import json, os, uuid
import nltk
import pyodbc
import time
from sqlalchemy import create_engine, event
from nltk import tokenize
from string import punctuation
from nltk.tokenize import PunktSentenceTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
# carrega os dados para aprendizado supervisionado
resenhas = pd.read_csv('imdb-reviews-pt-br.csv')

In [19]:
resenhas

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [20]:
# ajustando os sentimentos para 0/1
resenhas['classificacao'] = resenhas['sentiment'].replace(['neg','pos'], [0,1])

In [21]:
# identificando as palavras irrelevantes (stop words)
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')

In [22]:
# listando a pontuação
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)
    
palavras_irrelevantes_e_pontuacao = pontuacao + palavras_irrelevantes

In [23]:
# adicionando casos onde as aspas (simples e duplas) estão no final da frase
palavras_irrelevantes_e_pontuacao.append('".')
palavras_irrelevantes_e_pontuacao.append("'.")
palavras_irrelevantes_e_pontuacao.append('...')

In [24]:
# removendo os acentos
palavras_irrelevantes_e_pontuacao_sem_acento = [unidecode.unidecode(sw) for sw in palavras_irrelevantes_e_pontuacao]

In [36]:
# tokenizador de frases
punct_tokenizer = tokenize.WordPunctTokenizer()

In [37]:
# calculador do stemm das palavras
stemmer = nltk.RSLPStemmer()

In [38]:
# ajustar as opinioes para o que se espera
# reduzindo palavras para o seu stemm; cuidado: bem lento
frases_processadas = list()
for opiniao in resenhas.text_pt:
    nova_opiniao = list()
    palavras_opiniao = punct_tokenizer.tokenize(unidecode.unidecode(opiniao.lower()))
    for palavra in palavras_opiniao:
        if palavra not in palavras_irrelevantes_e_pontuacao_sem_acento:
            nova_opiniao.append(stemmer.stem(palavra))
    frases_processadas.append(' '.join(nova_opiniao))

resenhas['opiniao_tratada'] = frases_processadas

In [39]:
# define uma função para criar um classificador de textos, usando TF-IDF e ngrams 1-2
def classificar_texto_tfidf_ngrams(texto, coluna_texto, coluna_classificacao):
  
    vetorizador = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
    big_bag_of_words = vetorizador.fit_transform(texto[coluna_texto])

    X_train, X_test, Y_train, Y_test = train_test_split(big_bag_of_words,
                                                        texto[coluna_classificacao],
                                                        random_state = 42)

    regressao_logistica = LogisticRegression(solver='lbfgs')
    regressao_logistica.fit(X_train, Y_train)
    
    return regressao_logistica.score(X_test, Y_test), \
        regressao_logistica, \
        vetorizador


In [40]:
# executa a função para criar os objetos
acuracia, modelo, vetorizador = classificar_texto_tfidf_ngrams(resenhas, 'opiniao_tratada', 'classificacao')

In [41]:
print(acuracia)

0.8856449656287909


In [42]:
# simulador
def analise_sentimento(modelo, vetorizador, opiniao):
    nova_opiniao = list()
    palavras_opiniao = tokenize.WordPunctTokenizer().tokenize(unidecode.unidecode(opiniao.lower()))
    for palavra in palavras_opiniao:
        if palavra not in palavras_irrelevantes_e_pontuacao_sem_acento:
            nova_opiniao.append(stemmer.stem(palavra))
    opiniao_filtrada = ' '.join(nova_opiniao)
    print('Opinião limpa:', opiniao_filtrada)
    
    big_bag_of_words = vetorizador.transform([opiniao_filtrada])
#     print(big_bag_of_words)
    return modelo.predict(big_bag_of_words)[0] == 1

In [48]:

azureServer = "server-petz.database.windows.net"
azureDB = "db-petz"
userName = "ADM"
password = "FIAP1dtso"
driver = "{ODBC Driver 17 for SQL Server}"

connectionString = f"DRIVER={driver};SERVER={azureServer};PORT=1433;DATABASE={azureDB};UID={userName};PWD={password}"

conn = pyodbc.connect(connectionString)

cursor = conn.cursor()    



databaseConnection(driver, azureServer, azureDB, userName, password)
sql = f"SELECT * FROM TB_Reclame_Aqui_Respondidas"
df_reclamacoes = pd.read_sql(insert_to_tmp_tbl_stmt, conn)

#start = time.time() #Note start time
#cursor.execute(insert_to_tmp_tbl_stmt) #load data into azure sql db
#end = time.time() #Note end time

# print(f'{len(df_reclamacoes)} rows inserted in cursor.close() conn.close() table')
print(f'{(end - start)/60} minutes elapsed') 

conn.commit()
cursor.close()
conn.close()


0.00046813090642293295 minutes elapsed


C:\Users\henri\AppData\Local\Temp\ipykernel_29240\2072299128.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reclamacoes = pd.read_sql(insert_to_tmp_tbl_stmt, conn)


0.00026002724965413414 minutes elapsed


In [49]:
df_reclamacoes

,Status_Reclamacao,Local_Reclamacao,Data_Hora_Reclamacao,Reclamacao_id,Titulo_Reclamacao,Reclamacao_Comentario,Data_Resposta_Empresa,Resposta_Empresa,Link_Reclamacao
0,Resolvido,São Paulo - SP,18/10/2022 às 09:23,ID: 151982989,Problema com entrega,No dia 08/10/22 realizei a compra de um produt...,19/10/2022 às 13:53,"Protocolo: 221018-000868Olá, Cleber! Tudo bem?...",https://www.reclameaqui.com.br/petz/problema-c...
1,Resolvido,Parnamirim - RN,18/10/2022 às 06:52,ID: 151975757,Colchão gelado é quente.,Ontem comprei um colchão gelado pelo site e fu...,18/10/2022 às 18:22,"Olá, Anna!Tudo bem? ...",https://www.reclameaqui.com.br/petz/colchao-ge...
2,Resolvido,Campo Grande - MS,17/10/2022 às 14:42,ID: 151934775,Não informam opnde priduto está,Quando fiz o pagamento constava como produto d...,18/10/2022 às 17:54,Olá Diego.Boa tarde!Conforme contato através d...,https://www.reclameaqui.com.br/petz/nao-inform...
3,Resolvido,São Paulo - SP,15/10/2022 às 23:26,ID: 151871493,Mercadoria com frete expresso não entregue,Efetuei uma compra no site da petz no dia 14/1...,18/10/2022 às 12:34,"Olá, Sandra.Tudo bem?Pedimos sinceras desculpa...",https://www.reclameaqui.com.br/petz/mercadoria...
4,Resolvido,Brasília - DF,14/10/2022 às 08:46,ID: 151768565,Larvas na ração,Ocorre que comprei uma ração Guabi Natural Adu...,17/10/2022 às 10:39,Olá Alane.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/larvas-na-...
...,...,...,...,...,...,...,...,...,...
803,Resolvido,Recife - PE,25/06/2022 às 19:31,ID: 145727865,"Recebi ração caríssima, data validade vencida","Comprei uma Ração super premium, caríssima par...",29/06/2022 às 11:27,Olá Katia.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/recebi-rac...
804,Resolvido,Santos - SP,25/06/2022 às 18:31,ID: 145726207,F a l s a afirmação entrega expressa,Fiz o pedido 110386671 às 14:48 com entrega ex...,29/06/2022 às 11:12,Olá Gonzalo.Bom dia! Conforme contato telefôni...,https://www.reclameaqui.com.br/petz/f-a-l-s-a-...
805,Resolvido,Guarulhos - SP,25/06/2022 às 18:14,ID: 145725733,Sac não responde direito atendimento horrível,Liguei no Sac apenas para saber se um produto ...,30/06/2022 às 12:14,Olá Mariana.Boa tarde! Conforme contato telefô...,https://www.reclameaqui.com.br/petz/sac-nao-re...
806,Resolvido,Belo Horizonte - MG,25/06/2022 às 18:09,ID: 145725611,Demora na retirada,"Fiz comprar na Petz no dia 23/06/2022, onde es...",01/07/2022 às 09:59,"Olá Kaio.Bom dia! Conforme contato telefônico,...",https://www.reclameaqui.com.br/petz/demora-na-...


In [50]:
df_reclamacoes['sentimento'] = df_reclamacoes['Reclamacao_Comentario'].apply(lambda opiniao: opiniao_eh_positiva(modelo, vetorizador, opiniao))

Opinião limpa: dia 08 10 22 realiz compr produt raca cae const entreg 4 dia utel pag fret receb produt terc feir dia 11 10 22 receb sm empr uell transport inform entreg rot realiz aguard dia int porqu trabalh cas hom offic acontec entreg mand alert apen inform insucess tent entreg remarc dia 17 10 22 gast dinh compr raca aqu bairr pag car produt compr aguard confi esper par ana deix petz com poi acontec com compr aqu pens compr nad ped cheg descas client sempr compr confianc empr volt compr
Opinião limpa: ont compr colcha gel sit faz retir loj zon nort natal perceb colcha faz barulh estranh dentr estal fic gel contrari permanec temp tod quent cachorr fic cim esquent aind li questa refrescanc dad pes animal cim colcha cachorr deit colcha peg fog gost efetu cancel compr
Opinião limpa: fiz pag const produt disponi retir 1 hor loj aprov surg aguard 3 6 dia entr contat chat ne telefon cai ligaca ning inform ond produt
Opinião limpa: efet compr sit petz dia 14 10 22 fatur autor carta 11 35 p

Opinião limpa: compr 4 pacot rac premi rac especif bulldog send raco consum dentr praz 4 mes pet com sac dia val lembr sempr petz sempr aproveit promoco sit assim armazen raca petz armazen cas acontec pacot abr chei larv petz tir respons troc inform praz 30 dia troc val aind ressalt fal troc fal vici ocult produt abr tod raco confer atend protocol 221014000309 simples rud diss inform pront sab vontad faz compr nov pacot not fiscal atual us dat troc quer raca troc palhac atend pess melhor
Opinião limpa: fiz compr dia 04 10 gaiol ferplast ped 128176639 ), coloq retir loj poi promet acontec 3 4 dia utel press poi bem sext dia 7 inform praz precis alter est disponi retir dia 11 10 entr contat loj dia 11 poi liberaca ocorr fal final dia receb mens ir busc hoj 13 10 entr contat nov poi nad liber atend ped aguard amanh dia 14 possi retir situaca pass tod limit compr retir loj just ter press quer produt continu assim ire acion procon
Opinião limpa: fiz pag const produt disponi retir 1 hor loj 

Opinião limpa: tent resolv problem empr colabor compr aliment atrav sistem entreg rapp hamst marc petz embal const informaca pet adapt aliment fabric devolv dinh problem tent faz devoluca produt us canal disponi sit diss troc feit rapp entr contat rapp inform apen plataform entreg possu vincul nenhum marc mei obvi ), assim atend petz continu aleg devoca feit rapp enfim ta complic empr colab cumpr diz embal produt
Opinião limpa: dia 08 10 22 realiz compr produt raca cae const entreg 4 dia utel pag fret receb produt terc feir dia 11 10 22 receb sm empr uell transport inform entreg rot realiz aguard dia int porqu trabalh cas hom offic acontec entreg mand alert apen inform insucess tent entreg remarc dia 17 10 22 gast dinh compr raca aqu bairr pag car produt compr aguard confi esper par ana deix petz com poi acontec com compr aqu pens compr nad ped cheg descas client sempr compr confianc empr volt compr
Opinião limpa: ont compr colcha gel sit faz retir loj zon nort natal perceb colcha faz 

Opinião limpa: vari gat enta sei quant temp dur estoqu raca progr pra faz ped pra dar temp cheg acontec fiz ped dia 26 09 8 dia utel entreg entr contat chat posicion quant entreg qualqu cois tip apen diss entr contat comig nad agor client assidu tant loj onlin quant fisic ach tamanh descas ness mei temp fiz compr 3 raco retir 45 minut cheg cas abr gost entr contat pra troc outr dua fech inform troc it ped apen tod it vez alg revist ne petz direit arrepend 7 dia acord cdc
Opinião limpa: fiz compr ont dua raco vermifug antipulg ped entreg hoj apo 19h port moment verific ped dentr apart perceb falt it cheg 3 3 pacot falt it lig quatr vez loj despach ped nenhum ligaco atend falt produt simparic atend loj onlin soment 16h gost ped envi ped 129035994
Opinião limpa: hoj fiz compr onlin part manh tend praz entreg 1 dia util compr hav medicaca pod alter pra 1 seman produt consum imediat ???? receb comunic part tard solicit cancel estorn absurd praz 1 seman estorn 1 vez problem empr ocorr junh v

Opinião limpa: fiz infeliz escolh faz assinat empr ant recorrenc produt tent divers vez cancel app sucess dia 03 10 09 54h lig fix inform atend mar assinat cancel receb inclusiv mail confirm cancel entretant contin receb tent cobranc carta credit lig nov hoj dia 05 10 13 40h inform atend brun assinat permanec ativ atend cancel soment usuari envi mail confirm cancel exist fiz nov tent cancel bendit assinat sit aparent sucess nenhum confianc ness empr envi mail confirm cancel efetiv
Opinião limpa: compr realiz aplic dia 03 10 2022 produt compr raca premi formul cae adult rac med sab frang 15kg produt entreg dia 04 10 2022 raca premi rac especif spitz alema filhot 01kg entr contat chat dia 04 10 2022 ped 48 hor respost deix clar pod abr produt err perd direit troc hoj lig atend atend afirm 50 produt pod consum poi err petz apo ligaca receb mail inform produt retir 7 dia utel apo poss demor 7 dia receb dinh volt totaliz 14 dia utel cachorr dev fic raca poi poss abr pacot acord mail trat re

...

Opinião limpa: realiz compr doi ped credit entreg apen alegaca outr ped falt estoqu send sit informaca disponibil produt estoqu alem diss aind receb nenhum mail sobr estorn carta it falt cobr ambos produt sid entreg falt informaco sit estoqu enta deix clar
Opinião limpa: feit compr dia 02 07 2022 numer ped ped 111553599 dad baix receb por pres moment receb nad .. prim vez acontec
Opinião limpa: realiz compr aplic petz dia 16 06 2022 praz retir loj 11 dia utel pod retir part dia 01 07 2022 entant retir liber hoj receb ped aplic aind const produt transferenc unidad tod vez lig petz informaca pass send dad prior final dia ped liber retir nunc fic disponi retir hoj 06 07 2022 nad ped liber muit falt respeit profission poi lig entr contat comig pra pass nenhum informaca simples enrol liber ped ped quas 500 real chate sempr compr petz fiq decepcion falt compromiss client quer lib ped retir
Opinião limpa: x reclam petz deu ceu fiz compr pra entreg guarulh mesm entreg artur alvim ond mor irm .

Opinião limpa: fiz compr hoj dia 02 07 22 pag entreg express dev lev 3 hor pra receb pacot 15kg raca entr aplic pra ver q acontec .. poi hav pass 5 hor ped qdo surpr const entreg entr atend app fal algu sobr err ning fal poi horari atend encerr hoj resum .. pet vao fic raca
Opinião limpa: som 3 ped realiz modal entreg express cumpr praz atras hor especific entreg vai cheg vai cheg ped cancel motiv serv pess func atend respost pront ajud
Opinião limpa: compr loj on lin petz compr bols canguru col gui col fic grand bols cab cachorr cidad loj vou quer troc prefir devolv lig pra empr fal sobr acontec moc fal opca recolh minh cas enta fal ir vim recolh dentr 7 dia faz 15 dia vei ning peg lig var vez atend fal vai vim vem to precis dinh pra compr outr favor resolv
Opinião limpa: pessoal boa tard empr petz loj onlin ped 109878821 dat compr 22 06 2022 seri problem equip logis ", segund vez ness ano empr conseg entreg produt praz estipul propr empr empr conseg cumpr praz entreg dar assistenc pe

Opinião limpa: fiz compr onlin petz fiz pag nunc receb mercad .. fiz reclamaca .. fic faz estorn .. nunc fiz .. absurd .. eh edit reclam aqu
Opinião limpa: dia 25 mai fiz compr sit caus err descont cancel refaz compr cancel mesm sit aparec cancel fiz nov val continu send cobr carta pag 2 parcel entr contat telefon nad feit
Opinião limpa: fiz compr app diss q entreg produt receb dev fic esper dia soluc problem falt respeig client contrataca entreg ma indol
Opinião limpa: fiz ped dia 28 mai dia 30 mai receb mail inform ped sid entreg ped nunc receb fiz abert cham chat aplic obtiv nenhum respost enta entr contat telefon receb enta mens mail ped baix indev nov tent envi realiz por dia receb mail inform nov dat entreg minut outr mens diz ped entreg entr contat nov inform verific acontec nunc obtiv respost sobr ped receb ped dinh volt compr aplic ir loj moment arrepend tod vez compr aplic problem entreg falt informaco prova compr loj dev tod experienc ruim entreg
Opinião limpa: realiz compr 

Opinião limpa: ped atras contat chat respond conseg informaca ped lig atend fal sistem atend precari loj port petz
Opinião limpa: acab solic 2 medic import trat dog mesm statu disponi estoqu opca retir loj praz 45 minut liberaca apo efetuaca pag praz liberaca pul 3 dia utel informaca anteri ant pag total engan prejud direit consum faz trat seri cao petz inv ajud atrapalh solicit cancel telefon esper menos estorn agil ness empr falt transparenc sla engan afim atra client final afast gost client detra consegu 1 n ped 110829036n atend 220628002276
Opinião limpa: sempr fac compr raca pet perceb 120 dia encontr produt golden seni compr
Opinião limpa: boa tard fiz assinat petz mant tranquil compr raca por apo ter fatur raca dat program petz tod dia deix pres cas esper entreg final dia env mens diz imprevist pud entreg desd dia 22 06 tend compr raca p deix pet fom perceb red soc faz centen pesso muit reclamaco instagr
Opinião limpa: perd senh clic perd senh ped cpf coloc aparec msg diz cpf en

Opinião limpa: engan coloq medic assinat automa consig retir carta cadastr agor cheg mail fal medic pront retir gost tant assinat quant carta compr cancel
Opinião limpa: venh dat hoj 27 06 demonstr total indignaca insatisfaca empr pet compr 230 72 mercad dia 22 06 internet hoj 27 06 entreg praz internet 24hr entr contat resolv problem cachorr send aliment pequen quant desd enta porqu hoj cheg condico compr outr lug porqu val poderi gast gast pet esper trag raco algum form compens dia ter gast outr cois pra tent san fom cachorr numer ped 109879045protocol 220627000141
Opinião limpa: fiz ped aplic petz dia 23 06 2022 remedi cachorr precis tod dia outr limp bucal fiz ped poi diz 1 dia util alg cachorr com tod dia compr porqu urgent consig cancel petz !!! necessit quer nunc cont petz nad
Opinião limpa: dia 04 05 2022 realiz quer compr raca val r 154 90 contud hav compr mesm raca dia 02 05 2022 mod soliceit imediat cancel ped send inform ped hav sid cancel receb estorn carta credit entretan

In [51]:
df_reclamacoes

,Status_Reclamacao,Local_Reclamacao,Data_Hora_Reclamacao,Reclamacao_id,Titulo_Reclamacao,Reclamacao_Comentario,Data_Resposta_Empresa,Resposta_Empresa,Link_Reclamacao,sentimento
0,Resolvido,São Paulo - SP,18/10/2022 às 09:23,ID: 151982989,Problema com entrega,No dia 08/10/22 realizei a compra de um produt...,19/10/2022 às 13:53,"Protocolo: 221018-000868Olá, Cleber! Tudo bem?...",https://www.reclameaqui.com.br/petz/problema-c...,False
1,Resolvido,Parnamirim - RN,18/10/2022 às 06:52,ID: 151975757,Colchão gelado é quente.,Ontem comprei um colchão gelado pelo site e fu...,18/10/2022 às 18:22,"Olá, Anna!Tudo bem? ...",https://www.reclameaqui.com.br/petz/colchao-ge...,True
2,Resolvido,Campo Grande - MS,17/10/2022 às 14:42,ID: 151934775,Não informam opnde priduto está,Quando fiz o pagamento constava como produto d...,18/10/2022 às 17:54,Olá Diego.Boa tarde!Conforme contato através d...,https://www.reclameaqui.com.br/petz/nao-inform...,False
3,Resolvido,São Paulo - SP,15/10/2022 às 23:26,ID: 151871493,Mercadoria com frete expresso não entregue,Efetuei uma compra no site da petz no dia 14/1...,18/10/2022 às 12:34,"Olá, Sandra.Tudo bem?Pedimos sinceras desculpa...",https://www.reclameaqui.com.br/petz/mercadoria...,True
4,Resolvido,Brasília - DF,14/10/2022 às 08:46,ID: 151768565,Larvas na ração,Ocorre que comprei uma ração Guabi Natural Adu...,17/10/2022 às 10:39,Olá Alane.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/larvas-na-...,True
...,...,...,...,...,...,...,...,...,...,...
803,Resolvido,Recife - PE,25/06/2022 às 19:31,ID: 145727865,"Recebi ração caríssima, data validade vencida","Comprei uma Ração super premium, caríssima par...",29/06/2022 às 11:27,Olá Katia.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/recebi-rac...,True
804,Resolvido,Santos - SP,25/06/2022 às 18:31,ID: 145726207,F a l s a afirmação entrega expressa,Fiz o pedido 110386671 às 14:48 com entrega ex...,29/06/2022 às 11:12,Olá Gonzalo.Bom dia! Conforme contato telefôni...,https://www.reclameaqui.com.br/petz/f-a-l-s-a-...,True
805,Resolvido,Guarulhos - SP,25/06/2022 às 18:14,ID: 145725733,Sac não responde direito atendimento horrível,Liguei no Sac apenas para saber se um produto ...,30/06/2022 às 12:14,Olá Mariana.Boa tarde! Conforme contato telefô...,https://www.reclameaqui.com.br/petz/sac-nao-re...,False
806,Resolvido,Belo Horizonte - MG,25/06/2022 às 18:09,ID: 145725611,Demora na retirada,"Fiz comprar na Petz no dia 23/06/2022, onde es...",01/07/2022 às 09:59,"Olá Kaio.Bom dia! Conforme contato telefônico,...",https://www.reclameaqui.com.br/petz/demora-na-...,False


In [52]:
# ajustando label do output do modelo False =  Negativo -- True = Positivo
df_reclamacoes['sentimento'] = df_reclamacoes['sentimento'].replace([True,False], ['Positivo','Negativo'])

In [53]:
df_reclamacoes

,Status_Reclamacao,Local_Reclamacao,Data_Hora_Reclamacao,Reclamacao_id,Titulo_Reclamacao,Reclamacao_Comentario,Data_Resposta_Empresa,Resposta_Empresa,Link_Reclamacao,sentimento
0,Resolvido,São Paulo - SP,18/10/2022 às 09:23,ID: 151982989,Problema com entrega,No dia 08/10/22 realizei a compra de um produt...,19/10/2022 às 13:53,"Protocolo: 221018-000868Olá, Cleber! Tudo bem?...",https://www.reclameaqui.com.br/petz/problema-c...,Negativo
1,Resolvido,Parnamirim - RN,18/10/2022 às 06:52,ID: 151975757,Colchão gelado é quente.,Ontem comprei um colchão gelado pelo site e fu...,18/10/2022 às 18:22,"Olá, Anna!Tudo bem? ...",https://www.reclameaqui.com.br/petz/colchao-ge...,Positivo
2,Resolvido,Campo Grande - MS,17/10/2022 às 14:42,ID: 151934775,Não informam opnde priduto está,Quando fiz o pagamento constava como produto d...,18/10/2022 às 17:54,Olá Diego.Boa tarde!Conforme contato através d...,https://www.reclameaqui.com.br/petz/nao-inform...,Negativo
3,Resolvido,São Paulo - SP,15/10/2022 às 23:26,ID: 151871493,Mercadoria com frete expresso não entregue,Efetuei uma compra no site da petz no dia 14/1...,18/10/2022 às 12:34,"Olá, Sandra.Tudo bem?Pedimos sinceras desculpa...",https://www.reclameaqui.com.br/petz/mercadoria...,Positivo
4,Resolvido,Brasília - DF,14/10/2022 às 08:46,ID: 151768565,Larvas na ração,Ocorre que comprei uma ração Guabi Natural Adu...,17/10/2022 às 10:39,Olá Alane.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/larvas-na-...,Positivo
...,...,...,...,...,...,...,...,...,...,...
803,Resolvido,Recife - PE,25/06/2022 às 19:31,ID: 145727865,"Recebi ração caríssima, data validade vencida","Comprei uma Ração super premium, caríssima par...",29/06/2022 às 11:27,Olá Katia.Bom dia! Conforme contato através da...,https://www.reclameaqui.com.br/petz/recebi-rac...,Positivo
804,Resolvido,Santos - SP,25/06/2022 às 18:31,ID: 145726207,F a l s a afirmação entrega expressa,Fiz o pedido 110386671 às 14:48 com entrega ex...,29/06/2022 às 11:12,Olá Gonzalo.Bom dia! Conforme contato telefôni...,https://www.reclameaqui.com.br/petz/f-a-l-s-a-...,Positivo
805,Resolvido,Guarulhos - SP,25/06/2022 às 18:14,ID: 145725733,Sac não responde direito atendimento horrível,Liguei no Sac apenas para saber se um produto ...,30/06/2022 às 12:14,Olá Mariana.Boa tarde! Conforme contato telefô...,https://www.reclameaqui.com.br/petz/sac-nao-re...,Negativo
806,Resolvido,Belo Horizonte - MG,25/06/2022 às 18:09,ID: 145725611,Demora na retirada,"Fiz comprar na Petz no dia 23/06/2022, onde es...",01/07/2022 às 09:59,"Olá Kaio.Bom dia! Conforme contato telefônico,...",https://www.reclameaqui.com.br/petz/demora-na-...,Negativo


# Ingestão de um dataframe no banco de dados SQL Azure

In [56]:
def databaseConnection(driver, server, database, user, pwd):
    connectionString = f"DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={user};PWD={pwd}"
    conn = pyodbc.connect(connectionString)
    cursor = conn.cursor()    
    insert_to_tmp_tbl_stmt = f"INSERT INTO TB_Reclame_Aqui_Respondidas_Processada \
                    (Status_Reclamacao, Local_Reclamacao, Data_Hora_Reclamacao,Reclamacao_id, Titulo_Reclamacao, Reclamacao_Comentario, Data_Resposta_Empresa, Resposta_Empresa, Link_Reclamacao,Sentimento) \
                    values(?,?,?,?,?,?,?,?,?,?)"

    start = time.time() #Note start time
    cursor.executemany(insert_to_tmp_tbl_stmt, df_reclamacoes.values.tolist()) #load data into azure sql db
    end = time.time() #Note end time

    print(f'{len(df_reclamacoes)} rows inserted in cursor.close() conn.close() table')
    print(f'{(end - start)/60} minutes elapsed') 
    
    conn.commit()
    cursor.close()
    conn.close()
        
def main():
    azureServer = "server-petz.database.windows.net"
    azureDB = "db-petz"
    userName = "ADM"
    password = "FIAP1dtso"
    driver = "{ODBC Driver 17 for SQL Server}"
    databaseConnection(driver, azureServer, azureDB, userName, password)
    
if __name__ == '__main__':
    main()

808 rows inserted in cursor.close() conn.close() table
0.32687859535217284 minutes elapsed
